## 39. 객체를 제너릭하게 꾸성하려면 @classmethod를 통한 다형성을 활용하라

파이썬에서는 객체뿐만 아니라 클래스도 다형성을 지원한다.

다형성을 사용하면 계층을 이루는 여러 클래스가 자신에게 맞는 유일한 메서드 버전을 구현할 수 있다.

이는 같은 인터페이스를 만족하거나 같은 추상 기반 클래스를 공유하는 많은 클래스가 서로 다른 기능을 제공할 수 있다는 뜻이다.

예를 들어 맵리듀스 구현을 하나 작성하는데 입력 데이터를 표현할 수 있는 공통 클래스가 필요하다가 하자.

다음 코드는 이럴 때 사용하기 위해 정의한, 하위 클래스에서 다시 정의해야만 하는 read 메서드가 들어 있는 공통 클래스를 보여준다.

In [1]:
class InputData:
    def read(self):
        raise NotImplementedError

In [2]:
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        with open(self.path) as f:
            return f.read()

하위 클래스를 만들 수 있고, 하위 클래스는 처리할 데이터를 돌려주는 공통 read 인터페이스를 구현해야 한다.

비슷한 방법으로, 이 입력 데이터를 소비하는 공통 방법을 제공하는 맵 리듀스 작업자로 쓸 수 있는 추상 인터페이스를 정의하고 싶다.

In [3]:
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
    
    def map(self):
        raise NotImplementedError
    
    def reduce(self):
        raise NotImplementedError

다음 코드는 하위 클래스다.

In [4]:
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
        
    def reduce(self, other):
        self.result += other.result

이 구현은 아주 잘 작동할 것처럼 보이지만 각 부분을 연결하는 것이 문제다.

인터페이스와 추상화를 제공하는 멋진 클래스를 만들었지만 객체를 생성해 활용해야만 모든 클래스가 쓸모 있다.

가장 간단한 방법은 도우미 함수를 활용해 객체를 만들고 연결한다.

In [5]:
import os

def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))

다음으로 방금 generate_inputs를 통해 만드는 InputData 인스턴스들을 사용하는 LineCountWorker 인스턴스를 만든다.

In [6]:
def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

이 Worker 인스턴스의 map 단계를 여러 스레드에 공급해서 실행할 수 있다.

그 후 reduce를 반복적으로 호출해서 결과를 최종 값으로 합칠 수 있다.

In [7]:
from threading import Thread


def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()

    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

몇 가지 입력 파일을 대상으로 이 함수를 실행해보면 아주 훌륭하게 작동한다.

In [8]:
import os
import random

def write_test_files(tmpdir):
    os.makedirs(tmpdir)
    for i in range(100):
        with open(os.path.join(tmpdir, str(i)), 'w') as f:
            f.write('\n' * random.randint(0, 100))

tmpdir = 'test_inputs'
write_test_files(tmpdir)

result = mapreduce(tmpdir)
print(f'총 {result} 줄이 있습니다.')

총 4818 줄이 있습니다.


하지만 문제가 존재한다. 함수가 전혀 제너릭을 사용하지 않는다.

다른 InputData나 Worker 하위 클래스를 사용하고 싶다면 각 하위 클래스에 맞게 generate_inputs, create_workers, mapreduce를 재작성해야 한다.

파이썬에서는 생성자 메서드가 \_\_init\_\_ 밖에 없다는게 문제다.

이 ㅁ누제를 해결하는 가장 좋은 방법은 클래스 메서드 다형성을 사용하는 것이다.

이 방식은 InputData.read에서 사용했던 인스턴스 메서드의 다형성과 똑같은데, 클래스로 만들어낸 개별 객체에 적용되지 않고 클래스 전체에 적용된다는 점만 다르다.

In [9]:
class GenericInputData:
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

In [10]:
class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

In [11]:
class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

이 코드에서 input_class.generate_inputs 호출이 바로 여기서 보여주려는 클래스 다형성의 예이다.

In [12]:
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')

    def reduce(self, other):
        self.result += other.result

In [13]:
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

In [14]:
config = {'data_dir': tmpdir}
result = mapreduce(LineCountWorker, PathInputData, config)
print(f'총 {result} 줄이 있습니다.')

Exception in thread Thread-176:
Traceback (most recent call last):
  File "/Users/Danny/.pyenv/versions/3.8.5/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/Danny/.pyenv/versions/3.8.5/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-14df73dd467d>", line 3, in map
  File "<ipython-input-10-0e33911915a1>", line 7, in read
IsADirectoryError: [Errno 21] Is a directory: 'test_inputs/.ipynb_checkpoints'


TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'

## 기억해야 할 내용
- 파이썬의 클래스에는 생성자가 \_\_init\_\_ 메서드 뿐이다.
- @classmethod를 사용하면 클래스에 다른 생성자를 정의할 수 있다.
- 클래스 메서드 다형성을 활용하면 여러 구체적인 하위 클래스의 객체를 만들고 연결하는 제너릭한 방법을 제공할 수 있다.